In [10]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib qt

# Define the regions and conditions for plotting the balls
OldRegStartSS_Y = 189
OldRegStumpHeightSS_Y = 312
OldRegStumpLineSS_Y = 406
OldRegEndSS_Y = 446
OldRegStartSS_X = 144
OldRegEndSS_X = 462

RegStartSS_Y = 90
RegEndSS_Y = 467
RegStartSS_X = 300
RegEndSS_X = 781
RegStumpHeightSS_Y = 335
RegStumpLineSS_Y = 439

width = 0
height = 0

def ShortZoneXaxis(HeightX):
    if HeightX >= OldRegStartSS_X and HeightX <= OldRegEndSS_X:
        return (((RegEndSS_X - RegStartSS_X) / (OldRegEndSS_X - OldRegStartSS_X)) * (HeightX - OldRegStartSS_X)) + RegStartSS_X
    else:
        return None

def ShortZoneYaxis(HeightY):
    if HeightY >= OldRegStartSS_Y and HeightY <= OldRegStumpHeightSS_Y:
        return (((RegStumpHeightSS_Y - RegStartSS_Y) / (OldRegStumpHeightSS_Y - OldRegStartSS_Y)) * (HeightY - OldRegStartSS_Y)) + RegStartSS_Y
    elif HeightY > OldRegStumpHeightSS_Y and HeightY <= OldRegStumpLineSS_Y:
        return (((RegStumpLineSS_Y - RegStumpHeightSS_Y) / (OldRegStumpLineSS_Y - OldRegStumpHeightSS_Y)) * (HeightY - OldRegStumpHeightSS_Y)) + RegStumpHeightSS_Y
    elif HeightY > OldRegStumpLineSS_Y and HeightY <= OldRegEndSS_Y:
        return (((RegEndSS_Y - RegStumpLineSS_Y) / (OldRegEndSS_Y - OldRegStumpLineSS_Y)) * (HeightY - OldRegStumpLineSS_Y)) + RegStumpLineSS_Y
    else:
        return None

def plot_balls(x, y, StrikerBattingType, ball_type):
    XAxisValue_New = ShortZoneXaxis(x)
    YAxisValue_New = ShortZoneYaxis(y)
    if XAxisValue_New is not None and YAxisValue_New is not None:
        X = (XAxisValue_New / 1080 * width) - 7
        Y = (YAxisValue_New / 600 * height) - 13
        color = get_ball_color(ball_type)
        plt.scatter(X, Y, marker='o', color=color, label=ball_type)

def get_ball_color(ball_type):
    colors = {
        '1s': 'goldenrod',
        '2s': 'blue',
        '3s': 'green',
        '0s': 'black',
        'Batwkts': 'azure',
        '4s': 'darkblue',
        '6s': 'red'
    }
    return colors.get(ball_type, 'gray')  # Default color for unknown types

def determine_ball_type(row):
    if row['1s'] == 1:
        return '1s'
    elif row['2s'] == 1:
        return '2s'
    elif row['3s'] == 1:
        return '3s'
    elif row['0s'] == 1:
        return '0s'
    elif row['Batwkts'] == 1:
        return 'Batwkts'
    elif row['4s'] == 1:
        return '4s'
    elif row['6s'] == 1:
        return '6s'
    else:
        return None

image_paths = {
        '1': 'bee_r.jpg',
        '2': 'bee_l.jpg'
}

# Read the CSV file containing x, y, battingtype, StrikerName, and runs columns
data = pd.read_csv('NewData.csv')  # Update with your CSV file path
player_name_input = input("Enter the name of the player: ")
# Ask user for type of bowler (Pace, Spin, or Both)
bowler_type_input = input("Enter the type of bowler (Pace:1, Spin:2, or Both:3): ")

# Get the data based on the bowler type
if bowler_type_input == "1":
    bowler_data = data[data['PaceorSpin'] == 1]
elif bowler_type_input == "2":
    bowler_data = data[data['PaceorSpin'] == 2]
elif bowler_type_input== "3":
    bowler_data = data
else:
    print("Invalid input. Please enter 'Pace', 'Spin', or 'Both'.")
    exit()

# Get the unique player names and batting types from the bowler data
player_names = bowler_data['StrikerName'].unique()

# Check if the player name is in the bowler data
if player_name_input in player_names:
    # Filter the data for the specified player
    player_data = bowler_data[bowler_data['StrikerName'] == player_name_input]
    
    # Get the unique batting types for the specified player
    battingtypes = player_data['StrikerBattingType'].unique()
    
    # Plot balls based on x, y, and battingtype values for the specified player
    for battingtype in battingtypes:
        # Load the appropriate image based on battingtype
        beehive_img = plt.imread(image_paths[str(battingtype)])
        
        # Get the dimensions of the image
        height, width, _ = beehive_img.shape
    
        # Plot the beehive image
        plt.imshow(beehive_img)
        
        # Remove axis ticks, tick labels, and grid lines
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
    
        # Plot balls for the specified player and battingtype
        subset_data = player_data[player_data['StrikerBattingType'] == battingtype]
        for index, row in subset_data.iterrows():
            ball_type = determine_ball_type(row)  # Determine the ball type
            if ball_type is not None:
                plot_balls(row['HeightX'], row['HeightY'], row['StrikerBattingType'], ball_type)
    
         # Show the legend
        legend_elements = [
                plt.Line2D([0], [0], marker='o', color='w', label='0s', markerfacecolor='black', markersize=10),
                plt.Line2D([0], [0], marker='o', color='w', label='1s', markerfacecolor='goldenrod', markersize=10),
                plt.Line2D([0], [0], marker='o', color='w', label='2s', markerfacecolor='blue', markersize=10),
                plt.Line2D([0], [0], marker='o', color='w', label='3s', markerfacecolor='green', markersize=10),
                plt.Line2D([0], [0], marker='o', color='w', label='4s', markerfacecolor='darkblue', markersize=10),
                plt.Line2D([0], [0], marker='o', color='w', label='6s', markerfacecolor='red', markersize=10),
                plt.Line2D([0], [0], marker='o', color='w', label='Wickets', markerfacecolor='azure', markersize=10),
            ]
        plt.legend(handles=legend_elements, loc='lower center', bbox_to_anchor=(0.5, 0.05), ncol=7, prop={'size':8})

        # Show the plot
        plt.title(f"Beehive Plot for {player_name_input} ")
        plt.show()
else:
    print("Player name not found in the data.")


In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

# Define the regions and conditions for plotting the balls
OldRegStartSS_Y = 189
OldRegStumpHeightSS_Y = 312
OldRegStumpLineSS_Y = 406
OldRegEndSS_Y = 446
OldRegStartSS_X = 144
OldRegEndSS_X = 462

RegStartSS_Y = 90
RegEndSS_Y = 467
RegStartSS_X = 300
RegEndSS_X = 781
RegStumpHeightSS_Y = 335
RegStumpLineSS_Y = 439

width = 0
height = 0

def ShortZoneXaxis(HeightX):
    if HeightX >= OldRegStartSS_X and HeightX <= OldRegEndSS_X:
        return (((RegEndSS_X - RegStartSS_X) / (OldRegEndSS_X - OldRegStartSS_X)) * (HeightX - OldRegStartSS_X)) + RegStartSS_X
    else:
        return None

def ShortZoneYaxis(HeightY):
    if HeightY >= OldRegStartSS_Y and HeightY <= OldRegStumpHeightSS_Y:
        return (((RegStumpHeightSS_Y - RegStartSS_Y) / (OldRegStumpHeightSS_Y - OldRegStartSS_Y)) * (HeightY - OldRegStartSS_Y)) + RegStartSS_Y
    elif HeightY > OldRegStumpHeightSS_Y and HeightY <= OldRegStumpLineSS_Y:
        return (((RegStumpLineSS_Y - RegStumpHeightSS_Y) / (OldRegStumpLineSS_Y - OldRegStumpHeightSS_Y)) * (HeightY - OldRegStumpHeightSS_Y)) + RegStumpHeightSS_Y
    elif HeightY > OldRegStumpLineSS_Y and HeightY <= OldRegEndSS_Y:
        return (((RegEndSS_Y - RegStumpLineSS_Y) / (OldRegEndSS_Y - OldRegStumpLineSS_Y)) * (HeightY - OldRegStumpLineSS_Y)) + RegStumpLineSS_Y
    else:
        return None

def plot_balls(x, y,StrikerBattingType, ball_type):
    XAxisValue_New = ShortZoneXaxis(x)
    YAxisValue_New = ShortZoneYaxis(y)
    if XAxisValue_New is not None and YAxisValue_New is not None:
        X = (XAxisValue_New / 1080 * width) - 7
        Y = (YAxisValue_New / 600 * height) - 13
        color = get_ball_color(ball_type)
        plt.scatter(X, Y, marker='o', color=color, label=ball_type)

def get_ball_color(ball_type):
    colors = {
        '1s': 'goldenrod',
        '2s': 'blue',
        '3s': 'green',
        '0s': 'black',
        'Batwkts': 'azure',
        '4s': 'darkblue',
        '6s': 'red'
    }
    return colors.get(ball_type, 'gray')  # Default color for unknown types

def determine_ball_type(row):
    if row['1s'] == 1:
        return '1s'
    elif row['2s'] == 1:
        return '2s'
    elif row['3s'] == 1:
        return '3s'
    elif row['0s'] == 1:
        return '0s'
    elif row['Batwkts'] == 1:
        return 'Batwkts'
    elif row['4s'] == 1:
        return '4s'
    elif row['6s'] == 1:
        return '6s'
    else:
        return None

image_paths = {
        '1': 'bee_r.jpg',
        '2': 'bee_l.jpg'
}

# Read the CSV file containing x, y, battingtype, StrikerName, and runs columns
data = pd.read_csv('NewData.csv')  # Update with your CSV file path

# Date filter
data['Date'] = pd.to_datetime(data['date'], dayfirst=True)
start_date = pd.to_datetime(input("Enter start date (YYYY-MM-DD): "))
end_date = pd.to_datetime(input("Enter end date (YYYY-MM-DD): "))
filtered_data = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)]

# Competition filter
competitions = list(filtered_data['CompName'].unique())
selected_competition = input(f"Select competition ({', '.join(competitions)}): ")
if selected_competition:
    filtered_data = filtered_data[filtered_data['CompName'] == selected_competition]

# Batsman club filter
bat_club_names = list(map(str, filtered_data['battingclubid'].unique()))
selected_bat_club_name = input(f"Select the batsman's club id ({', '.join(bat_club_names)}): ")
if selected_bat_club_name:
    filtered_data = filtered_data[filtered_data['battingclubid'] == int(selected_bat_club_name)]

# Match filter
match_ids = ['All'] + list(map(str, filtered_data['matchid'].unique()))
selected_match_id = input(f"Select Match ({', '.join(match_ids)}): ")
if selected_match_id != 'All':
    filtered_data = filtered_data[filtered_data['matchid'] == int(selected_match_id)]

# Batsman filter
batsman_names = ['All'] + list(filtered_data['StrikerName'].unique())
selected_batsman_name = input(f"Select the batsman's name ({', '.join(batsman_names)}): ")
if selected_batsman_name != 'All':
    filtered_data = filtered_data[filtered_data['StrikerName'] == selected_batsman_name]

# Pace or Spin filter
pace_or_spin = input("Select bowler type (Pace, Spin, Both): ")
if pace_or_spin == 'Pace':
    filtered_data = filtered_data[filtered_data['PaceorSpin'] == 1]
elif pace_or_spin == 'Spin':
    filtered_data = filtered_data[filtered_data['PaceorSpin'] == 2]

# Bowling Type Group filter
if pace_or_spin == 'Pace':
    bowling_type_options = ["All", "RAP", "LAP"]
    selected_bowling_types = input(f"Select Bowling Type Group ({', '.join(bowling_type_options)}): ")
    if selected_bowling_types != "All":
        bowling_type_values = []
        if "RAP" in selected_bowling_types:
            bowling_type_values.append(1)
        if "LAP" in selected_bowling_types:
            bowling_type_values.append(2)
        filtered_data = filtered_data[filtered_data['BowlingTypeGroup'].isin(bowling_type_values)]
elif pace_or_spin == 'Spin':
    bowling_type_options = ["All", "RAO", "SLAO", "RALB", "LAC"]
    selected_bowling_types = input(f"Select Bowling Type Group ({', '.join(bowling_type_options)}): ")
    if selected_bowling_types != "All":
        bowling_type_values = []
        if "RAO" in selected_bowling_types:
            bowling_type_values.append(3)
        if "SLAO" in selected_bowling_types:
            bowling_type_values.append(4)
        if "RALB" in selected_bowling_types:
            bowling_type_values.append(5)
        if "LAC" in selected_bowling_types:
            bowling_type_values.append(6)
        filtered_data = filtered_data[filtered_data['BowlingTypeGroup'].isin(bowling_type_values)]

# Phase selection
phase_type = input("Select phase type (3Phase, 4Phase): ")
if phase_type == "3Phase":
    phase_options = ["All", 1, 2, 3]
    selected_phase = input(f"Select Phase ({', '.join(map(str, phase_options))}): ")
    if selected_phase != "All":
        filtered_data = filtered_data[filtered_data['Phase3idStar'] == int(selected_phase)]
elif phase_type == "4Phase":
    phase_options = ["All", 1, 2, 3, 4]
    selected_phase = input(f"Select Phase ({', '.join(map(str, phase_options))}): ")
    if selected_phase != "All":
        filtered_data = filtered_data[filtered_data['Phase4id'] == int(selected_phase)]

# Run types filter
run_types = input("Select run types (comma-separated, e.g., '1s, 4s, 6s'): ").split(',')
run_types = [run_type.strip() for run_type in run_types]

# Filter based on run types
for run_type in run_types:
    filtered_data = filtered_data[filtered_data[run_type] == 1]

# Ensure filtered_data is not empty before accessing its elements
if not filtered_data.empty:
    # Read the image corresponding to the batsman's batting type
    batsman_batting_type = filtered_data['StrikerBattingType'].iloc[0]
    image_path = image_paths[str(batsman_batting_type)]
    img = plt.imread(image_path)
    height, width, _ = img.shape

    plt.imshow(img)

    # Plot each ball in the filtered data
    for index, row in filtered_data.iterrows():
        ball_type = determine_ball_type(row)
        if ball_type:
            plot_balls(row['x'], row['y'], row['StrikerBattingType'], ball_type)

    plt.show()
else:
    print("No data available for the selected filters.")


#TESTING PT3

No data available for the selected filters.


In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

# Define the regions and conditions for plotting the balls
OldRegStartSS_Y = 189
OldRegStumpHeightSS_Y = 312
OldRegStumpLineSS_Y = 406
OldRegEndSS_Y = 446
OldRegStartSS_X = 144
OldRegEndSS_X = 462

RegStartSS_Y = 90
RegEndSS_Y = 467
RegStartSS_X = 300
RegEndSS_X = 781
RegStumpHeightSS_Y = 335
RegStumpLineSS_Y = 439

width = 0
height = 0

def ShortZoneXaxis(HeightX):
    if HeightX >= OldRegStartSS_X and HeightX <= OldRegEndSS_X:
        return (((RegEndSS_X - RegStartSS_X) / (OldRegEndSS_X - OldRegStartSS_X)) * (HeightX - OldRegStartSS_X)) + RegStartSS_X
    else:
        return None

def ShortZoneYaxis(HeightY):
    if HeightY >= OldRegStartSS_Y and HeightY <= OldRegStumpHeightSS_Y:
        return (((RegStumpHeightSS_Y - RegStartSS_Y) / (OldRegStumpHeightSS_Y - OldRegStartSS_Y)) * (HeightY - OldRegStartSS_Y)) + RegStartSS_Y
    elif HeightY > OldRegStumpHeightSS_Y and HeightY <= OldRegStumpLineSS_Y:
        return (((RegStumpLineSS_Y - RegStumpHeightSS_Y) / (OldRegStumpLineSS_Y - OldRegStumpHeightSS_Y)) * (HeightY - OldRegStumpHeightSS_Y)) + RegStumpHeightSS_Y
    elif HeightY > OldRegStumpLineSS_Y and HeightY <= OldRegEndSS_Y:
        return (((RegEndSS_Y - RegStumpLineSS_Y) / (OldRegEndSS_Y - OldRegStumpLineSS_Y)) * (HeightY - OldRegStumpLineSS_Y)) + RegStumpLineSS_Y
    else:
        return None

def plot_balls(x, y, StrikerBattingType, ball_type):
    XAxisValue_New = ShortZoneXaxis(x)
    YAxisValue_New = ShortZoneYaxis(y)
    if XAxisValue_New is not None and YAxisValue_New is not None:
        X = (XAxisValue_New / 1080 * width) - 7
        Y = (YAxisValue_New / 600 * height) - 13
        color = get_ball_color(ball_type)
        plt.scatter(X, Y, marker='o', color=color, label=ball_type)

def get_ball_color(ball_type):
    colors = {
        '1s': 'goldenrod',
        '2s': 'blue',
        '3s': 'green',
        '0s': 'black',
        'Batwkts': 'azure',
        '4s': 'darkblue',
        '6s': 'red'
    }
    return colors.get(ball_type, 'gray')  # Default color for unknown types

def determine_ball_type(row):
    if row['1s'] == 1:
        return '1s'
    elif row['2s'] == 1:
        return '2s'
    elif row['3s'] == 1:
        return '3s'
    elif row['0s'] == 1:
        return '0s'
    elif row['Batwkts'] == 1:
        return 'Batwkts'
    elif row['4s'] == 1:
        return '4s'
    elif row['6s'] == 1:
        return '6s'
    else:
        return None

image_paths = {
    '1': 'bee_r.jpg',
    '2': 'bee_l.jpg'
}

# Read the CSV file containing x, y, battingtype, StrikerName, and runs columns
data = pd.read_csv('NewData.csv')  # Update with your CSV file path

# Date filter
data['Date'] = pd.to_datetime(data['date'], dayfirst=True)
start_date = pd.to_datetime(input("Enter start date (YYYY-MM-DD): "))
end_date = pd.to_datetime(input("Enter end date (YYYY-MM-DD): "))
filtered_data = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)]
print(f"Data after date filter: {filtered_data.shape[0]} rows")

# Competition filter
competitions = list(filtered_data['CompName'].unique())
selected_competition = input(f"Select competition ({', '.join(competitions)}): ")
if selected_competition:
    filtered_data = filtered_data[filtered_data['CompName'] == selected_competition]
print(f"Data after competition filter: {filtered_data.shape[0]} rows")

# Batsman club filter
bat_club_names = list(map(str, filtered_data['battingclubid'].unique()))
selected_bat_club_name = input(f"Select the batsman's club id ({', '.join(bat_club_names)}): ")
if selected_bat_club_name:
    filtered_data = filtered_data[filtered_data['battingclubid'] == int(selected_bat_club_name)]
print(f"Data after batsman club filter: {filtered_data.shape[0]} rows")

# Match filter
match_ids = ['All'] + list(map(str, filtered_data['matchid'].unique()))
selected_match_id = input(f"Select Match ({', '.join(match_ids)}): ")
if selected_match_id != 'All':
    filtered_data = filtered_data[filtered_data['matchid'] == int(selected_match_id)]
print(f"Data after match filter: {filtered_data.shape[0]} rows")

# Batsman filter
batsman_names = ['All'] + list(filtered_data['StrikerName'].unique())
selected_batsman_name = input(f"Select the batsman's name ({', '.join(batsman_names)}): ")
if selected_batsman_name != 'All':
    filtered_data = filtered_data[filtered_data['StrikerName'] == selected_batsman_name]
print(f"Data after batsman filter: {filtered_data.shape[0]} rows")

# Pace or Spin filter
pace_or_spin = input("Select bowler type (Pace, Spin, Both): ")
if pace_or_spin == 'Pace':
    filtered_data = filtered_data[filtered_data['PaceorSpin'] == 1]
elif pace_or_spin == 'Spin':
    filtered_data = filtered_data[filtered_data['PaceorSpin'] == 2]
print(f"Data after pace/spin filter: {filtered_data.shape[0]} rows")

# Bowling Type Group filter
if pace_or_spin == 'Pace':
    bowling_type_options = ["All", "RAP", "LAP"]
    selected_bowling_types = input(f"Select Bowling Type Group ({', '.join(bowling_type_options)}): ")
    if selected_bowling_types != "All":
        bowling_type_values = []
        if "RAP" in selected_bowling_types:
            bowling_type_values.append(1)
        if "LAP" in selected_bowling_types:
            bowling_type_values.append(2)
        filtered_data = filtered_data[filtered_data['BowlingTypeGroup'].isin(bowling_type_values)]
elif pace_or_spin == 'Spin':
    bowling_type_options = ["All", "RAO", "SLAO", "RALB", "LAC"]
    selected_bowling_types = input(f"Select Bowling Type Group ({', '.join(bowling_type_options)}): ")
    if selected_bowling_types != "All":
        bowling_type_values = []
        if "RAO" in selected_bowling_types:
            bowling_type_values.append(3)
        if "SLAO" in selected_bowling_types:
            bowling_type_values.append(4)
        if "RALB" in selected_bowling_types:
            bowling_type_values.append(5)
        if "LAC" in selected_bowling_types:
            bowling_type_values.append(6)
        filtered_data = filtered_data[filtered_data['BowlingTypeGroup'].isin(bowling_type_values)]
print(f"Data after bowling type group filter: {filtered_data.shape[0]} rows")

# Phase selection
phase_type = input("Select phase type (3Phase, 4Phase): ")
if phase_type == "3Phase":
    phase_options = ["All", 1, 2, 3]
    selected_phase = input(f"Select Phase ({', '.join(map(str, phase_options))}): ")
    if selected_phase != "All":
        filtered_data = filtered_data[filtered_data['Phase3idStar'] == int(selected_phase)]
elif phase_type == "4Phase":
    phase_options = ["All", 1, 2, 3, 4]
    selected_phase = input(f"Select Phase ({', '.join(map(str, phase_options))}): ")
    if selected_phase != "All":
        filtered_data = filtered_data[filtered_data['Phase4id'] == int(selected_phase)]
print(f"Data after phase filter: {filtered_data.shape[0]} rows")

# Run types filter
run_types = input("Select run types (comma-separated, e.g., '1s, 4s, 6s'): ").split(',')
run_types = [run_type.strip() for run_type in run_types]

# Filter based on run types
for run_type in run_types:
    filtered_data = filtered_data[filtered_data[run_type] == 1]
print(f"Data after run types filter: {filtered_data.shape[0]} rows")

# Ensure filtered_data is not empty before accessing its elements
if not filtered_data.empty:
    # Read the image corresponding to the batsman's batting type
    batsman_batting_type = filtered_data['StrikerBattingType'].iloc[0]
    image_path = image_paths[str(batsman_batting_type)]
    img = plt.imread(image_path)
    height, width, _ = img.shape

    plt.imshow(img)

    # Plot each ball in the filtered data
    for index, row in filtered_data.iterrows():
        ball_type = determine_ball_type(row)
        if ball_type:
            plot_balls(row['x'], row['y'], row['StrikerBattingType'], ball_type)

    plt.show()
else:
    print("No data available for the selected filters.")

#testing part3

Data after date filter: 3528 rows
Data after competition filter: 3528 rows
Data after batsman club filter: 640 rows
Data after match filter: 130 rows
Data after batsman filter: 3 rows
Data after pace/spin filter: 3 rows
Data after bowling type group filter: 3 rows
Data after phase filter: 3 rows
Data after run types filter: 0 rows
No data available for the selected filters.


In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

# Define the regions and conditions for plotting the balls
OldRegStartSS_Y = 189
OldRegStumpHeightSS_Y = 312
OldRegStumpLineSS_Y = 406
OldRegEndSS_Y = 446
OldRegStartSS_X = 144
OldRegEndSS_X = 462

RegStartSS_Y = 90
RegEndSS_Y = 467
RegStartSS_X = 300
RegEndSS_X = 781
RegStumpHeightSS_Y = 335
RegStumpLineSS_Y = 439

width = 0
height = 0

def ShortZoneXaxis(HeightX):
    if HeightX >= OldRegStartSS_X and HeightX <= OldRegEndSS_X:
        return (((RegEndSS_X - RegStartSS_X) / (OldRegEndSS_X - OldRegStartSS_X)) * (HeightX - OldRegStartSS_X)) + RegStartSS_X
    else:
        return None

def ShortZoneYaxis(HeightY):
    if HeightY >= OldRegStartSS_Y and HeightY <= OldRegStumpHeightSS_Y:
        return (((RegStumpHeightSS_Y - RegStartSS_Y) / (OldRegStumpHeightSS_Y - OldRegStartSS_Y)) * (HeightY - OldRegStartSS_Y)) + RegStartSS_Y
    elif HeightY > OldRegStumpHeightSS_Y and HeightY <= OldRegStumpLineSS_Y:
        return (((RegStumpLineSS_Y - RegStumpHeightSS_Y) / (OldRegStumpLineSS_Y - OldRegStumpHeightSS_Y)) * (HeightY - OldRegStumpHeightSS_Y)) + RegStumpHeightSS_Y
    elif HeightY > OldRegStumpLineSS_Y and HeightY <= OldRegEndSS_Y:
        return (((RegEndSS_Y - RegStumpLineSS_Y) / (OldRegEndSS_Y - OldRegStumpLineSS_Y)) * (HeightY - OldRegStumpLineSS_Y)) + RegStumpLineSS_Y
    else:
        return None

def plot_balls(x, y, StrikerBattingType, ball_type):
    XAxisValue_New = ShortZoneXaxis(x)
    YAxisValue_New = ShortZoneYaxis(y)
    if XAxisValue_New is not None and YAxisValue_New is not None:
        X = (XAxisValue_New / 1080 * width) - 7
        Y = (YAxisValue_New / 600 * height) - 13
        color = get_ball_color(ball_type)
        plt.scatter(X, Y, marker='o', color=color, label=ball_type)

def get_ball_color(ball_type):
    colors = {
        '1s': 'goldenrod',
        '2s': 'blue',
        '3s': 'green',
        '0s': 'black',
        'Batwkts': 'azure',
        '4s': 'darkblue',
        '6s': 'red'
    }
    return colors.get(ball_type, 'gray')  # Default color for unknown types

def determine_ball_type(row):
    if row['1s'] == 1:
        return '1s'
    elif row['2s'] == 1:
        return '2s'
    elif row['3s'] == 1:
        return '3s'
    elif row['0s'] == 1:
        return '0s'
    elif row['Batwkts'] == 1:
        return 'Batwkts'
    elif row['4s'] == 1:
        return '4s'
    elif row['6s'] == 1:
        return '6s'
    else:
        return None

image_paths = {
    '1': 'bee_r.jpg',
    '2': 'bee_l.jpg'
}

# Read the CSV file containing x, y, battingtype, StrikerName, and runs columns
data = pd.read_csv('NewData.csv')  # Update with your CSV file path

# Date filter
data['Date'] = pd.to_datetime(data['date'], dayfirst=True)
start_date = pd.to_datetime(input("Enter start date (YYYY-MM-DD): "))
end_date = pd.to_datetime(input("Enter end date (YYYY-MM-DD): "))
filtered_data = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)]
print(f"Data after date filter: {filtered_data.shape[0]} rows")

# Competition filter
competitions = list(filtered_data['CompName'].unique())
selected_competition = input(f"Select competition ({', '.join(competitions)}): ")
if selected_competition:
    filtered_data = filtered_data[filtered_data['CompName'] == selected_competition]
print(f"Data after competition filter: {filtered_data.shape[0]} rows")

# Batsman club filter
bat_club_names = list(map(str, filtered_data['battingclubid'].unique()))
selected_bat_club_name = input(f"Select the batsman's club id ({', '.join(bat_club_names)}): ")
if selected_bat_club_name:
    filtered_data = filtered_data[filtered_data['battingclubid'] == int(selected_bat_club_name)]
print(f"Data after batsman club filter: {filtered_data.shape[0]} rows")

# Match filter
match_ids = ['All'] + list(map(str, filtered_data['matchid'].unique()))
selected_match_id = input(f"Select Match ({', '.join(match_ids)}): ")
if selected_match_id != 'All':
    filtered_data = filtered_data[filtered_data['matchid'] == int(selected_match_id)]
print(f"Data after match filter: {filtered_data.shape[0]} rows")

# Batsman filter
batsman_names = ['All'] + list(filtered_data['StrikerName'].unique())
selected_batsman_name = input(f"Select the batsman's name ({', '.join(batsman_names)}): ")
if selected_batsman_name != 'All':
    filtered_data = filtered_data[filtered_data['StrikerName'] == selected_batsman_name]
print(f"Data after batsman filter: {filtered_data.shape[0]} rows")

# Pace or Spin filter
pace_or_spin = input("Select bowler type (Pace, Spin, Both): ")
if pace_or_spin == 'Pace':
    filtered_data = filtered_data[filtered_data['PaceorSpin'] == 1]
elif pace_or_spin == 'Spin':
    filtered_data = filtered_data[filtered_data['PaceorSpin'] == 2]
print(f"Data after pace/spin filter: {filtered_data.shape[0]} rows")

# Bowling Type Group filter
if pace_or_spin == 'Pace':
    bowling_type_options = ["All", "RAP", "LAP"]
    selected_bowling_types = input(f"Select Bowling Type Group ({', '.join(bowling_type_options)}): ")
    if selected_bowling_types != "All":
        bowling_type_values = []
        if "RAP" in selected_bowling_types:
            bowling_type_values.append(1)
        if "LAP" in selected_bowling_types:
            bowling_type_values.append(2)
        filtered_data = filtered_data[filtered_data['BowlingTypeGroup'].isin(bowling_type_values)]
elif pace_or_spin == 'Spin':
    bowling_type_options = ["All", "RAO", "SLAO", "RALB", "LAC"]
    selected_bowling_types = input(f"Select Bowling Type Group ({', '.join(bowling_type_options)}): ")
    if selected_bowling_types != "All":
        bowling_type_values = []
        if "RAO" in selected_bowling_types:
            bowling_type_values.append(3)
        if "SLAO" in selected_bowling_types:
            bowling_type_values.append(4)
        if "RALB" in selected_bowling_types:
            bowling_type_values.append(5)
        if "LAC" in selected_bowling_types:
            bowling_type_values.append(6)
        filtered_data = filtered_data[filtered_data['BowlingTypeGroup'].isin(bowling_type_values)]
print(f"Data after bowling type group filter: {filtered_data.shape[0]} rows")

# Phase selection
phase_type = input("Select phase type (3Phase, 4Phase): ")
if phase_type == "3Phase":
    phase_options = ["All", 1, 2, 3]
    selected_phase = input(f"Select Phase ({', '.join(map(str, phase_options))}): ")
    if selected_phase != "All":
        filtered_data = filtered_data[filtered_data['Phase3idStar'] == int(selected_phase)]
elif phase_type == "4Phase":
    phase_options = ["All", 1, 2, 3, 4]
    selected_phase = input(f"Select Phase ({', '.join(map(str, phase_options))}): ")
    if selected_phase != "All":
        filtered_data = filtered_data[filtered_data['Phase4id'] == int(selected_phase)]
print(f"Data after phase filter: {filtered_data.shape[0]} rows")

# Display unique values in run type columns before filtering
run_type_columns = ['1s', '2s', '3s', '0s', 'Batwkts', '4s', '6s']
for col in run_type_columns:
    unique_values = filtered_data[col].unique()
    print(f"Unique values in {col}: {unique_values}")

# Run types filter
run_types = input("Select run types (comma-separated, e.g., '1s, 4s, 6s'): ").split(',')
run_types = [run_type.strip() for run_type in run_types]

# Filter based on run types
for run_type in run_types:
    print(f"Filtering for run type: {run_type}")
    before_filtering = filtered_data.shape[0]
    filtered_data = filtered_data[filtered_data[run_type] == 1]
    after_filtering = filtered_data.shape[0]
    print(f"Rows before filtering: {before_filtering}, after filtering: {after_filtering}")

print(f"Data after run types filter: {filtered_data.shape[0]} rows")

# Ensure filtered_data is not empty before accessing its elements
if not filtered_data.empty:
    # Read the image corresponding to the batsman's batting type
    batsman_batting_type = filtered_data['StrikerBattingType'].iloc[0]
    image_path = image_paths[str(batsman_batting_type)]
    img = plt.imread(image_path)
    height, width, _ = img.shape

    plt.imshow(img)

    # Plot each ball in the filtered data
    for index, row in filtered_data.iterrows():
        ball_type = determine_ball_type(row)
        if ball_type:
            plot_balls(row['x'], row['y'], row['StrikerBattingType'], ball_type)

    plt.show()
else:
    print("No data available for the selected filters.")


Data after date filter: 3528 rows
Data after competition filter: 3528 rows
Data after batsman club filter: 650 rows
Data after match filter: 132 rows
Data after batsman filter: 31 rows
Data after pace/spin filter: 31 rows
Data after bowling type group filter: 31 rows
Data after phase filter: 31 rows
Unique values in 1s: [1 0]
Unique values in 2s: [0 1]
Unique values in 3s: [0]
Unique values in 0s: [0. 1.]
Unique values in Batwkts: [0 1]
Unique values in 4s: [0 1]
Unique values in 6s: [0 1]
Filtering for run type: 4s
Rows before filtering: 31, after filtering: 7
Filtering for run type: 6s
Rows before filtering: 7, after filtering: 0
Data after run types filter: 0 rows
No data available for the selected filters.


In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

# Define the regions and conditions for plotting the balls
OldRegStartSS_Y = 189
OldRegStumpHeightSS_Y = 312
OldRegStumpLineSS_Y = 406
OldRegEndSS_Y = 446
OldRegStartSS_X = 144
OldRegEndSS_X = 462

RegStartSS_Y = 90
RegEndSS_Y = 467
RegStartSS_X = 300
RegEndSS_X = 781
RegStumpHeightSS_Y = 335
RegStumpLineSS_Y = 439

width = 0
height = 0

def ShortZoneXaxis(HeightX):
    if HeightX >= OldRegStartSS_X and HeightX <= OldRegEndSS_X:
        return (((RegEndSS_X - RegStartSS_X) / (OldRegEndSS_X - OldRegStartSS_X)) * (HeightX - OldRegStartSS_X)) + RegStartSS_X
    else:
        return None

def ShortZoneYaxis(HeightY):
    if HeightY >= OldRegStartSS_Y and HeightY <= OldRegStumpHeightSS_Y:
        return (((RegStumpHeightSS_Y - RegStartSS_Y) / (OldRegStumpHeightSS_Y - OldRegStartSS_Y)) * (HeightY - OldRegStartSS_Y)) + RegStartSS_Y
    elif HeightY > OldRegStumpHeightSS_Y and HeightY <= OldRegStumpLineSS_Y:
        return (((RegStumpLineSS_Y - RegStumpHeightSS_Y) / (OldRegStumpLineSS_Y - OldRegStumpHeightSS_Y)) * (HeightY - OldRegStumpHeightSS_Y)) + RegStumpHeightSS_Y
    elif HeightY > OldRegStumpLineSS_Y and HeightY <= OldRegEndSS_Y:
        return (((RegEndSS_Y - RegStumpLineSS_Y) / (OldRegEndSS_Y - OldRegStumpLineSS_Y)) * (HeightY - OldRegStumpLineSS_Y)) + RegStumpLineSS_Y
    else:
        return None

def plot_balls(x, y, StrikerBattingType, ball_type):
    XAxisValue_New = ShortZoneXaxis(x)
    YAxisValue_New = ShortZoneYaxis(y)
    if XAxisValue_New is not None and YAxisValue_New is not None:
        X = (XAxisValue_New / 1080 * width) - 7
        Y = (YAxisValue_New / 600 * height) - 13
        color = get_ball_color(ball_type)
        plt.scatter(X, Y, marker='o', color=color, label=ball_type)

def get_ball_color(ball_type):
    colors = {
        '1s': 'goldenrod',
        '2s': 'blue',
        '3s': 'green',
        '0s': 'black',
        'Batwkts': 'azure',
        '4s': 'darkblue',
        '6s': 'red'
    }
    return colors.get(ball_type, 'gray')  # Default color for unknown types

def determine_ball_type(row):
    if row['1s'] == 1:
        return '1s'
    elif row['2s'] == 1:
        return '2s'
    elif row['3s'] == 1:
        return '3s'
    elif row['0s'] == 1:
        return '0s'
    elif row['Batwkts'] == 1:
        return 'Batwkts'
    elif row['4s'] == 1:
        return '4s'
    elif row['6s'] == 1:
        return '6s'
    else:
        return None

image_paths = {
    '1': 'bee_r.jpg',
    '2': 'bee_l.jpg'
}

# Read the CSV file containing x, y, battingtype, StrikerName, and runs columns
data = pd.read_csv('NewData.csv')  # Update with your CSV file path

# Date filter
data['Date'] = pd.to_datetime(data['date'], dayfirst=True)
start_date = pd.to_datetime(input("Enter start date (YYYY-MM-DD): "))
end_date = pd.to_datetime(input("Enter end date (YYYY-MM-DD): "))
filtered_data = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)]
print(f"Data after date filter: {filtered_data.shape[0]} rows")

# Competition filter
competitions = list(filtered_data['CompName'].unique())
selected_competition = input(f"Select competition ({', '.join(competitions)}): ")
if selected_competition:
    filtered_data = filtered_data[filtered_data['CompName'] == selected_competition]
print(f"Data after competition filter: {filtered_data.shape[0]} rows")

# Batsman club filter
bat_club_names = list(map(str, filtered_data['battingclubid'].unique()))
selected_bat_club_name = input(f"Select the batsman's club id ({', '.join(bat_club_names)}): ")
if selected_bat_club_name:
    filtered_data = filtered_data[filtered_data['battingclubid'] == int(selected_bat_club_name)]
print(f"Data after batsman club filter: {filtered_data.shape[0]} rows")

# Match filter
match_ids = ['All'] + list(map(str, filtered_data['matchid'].unique()))
selected_match_id = input(f"Select Match ({', '.join(match_ids)}): ")
if selected_match_id != 'All':
    filtered_data = filtered_data[filtered_data['matchid'] == int(selected_match_id)]
print(f"Data after match filter: {filtered_data.shape[0]} rows")

# Batsman filter
batsman_names = ['All'] + list(filtered_data['StrikerName'].unique())
selected_batsman_name = input(f"Select the batsman's name ({', '.join(batsman_names)}): ")
if selected_batsman_name != 'All':
    filtered_data = filtered_data[filtered_data['StrikerName'] == selected_batsman_name]
print(f"Data after batsman filter: {filtered_data.shape[0]} rows")

# Pace or Spin filter
pace_or_spin = input("Select bowler type (Pace, Spin, Both): ")
if pace_or_spin == 'Pace':
    filtered_data = filtered_data[filtered_data['PaceorSpin'] == 1]
elif pace_or_spin == 'Spin':
    filtered_data = filtered_data[filtered_data['PaceorSpin'] == 2]
print(f"Data after pace/spin filter: {filtered_data.shape[0]} rows")

# Bowling Type Group filter
if pace_or_spin == 'Pace':
    bowling_type_options = ["All", "RAP", "LAP"]
    selected_bowling_types = input(f"Select Bowling Type Group ({', '.join(bowling_type_options)}): ")
    if selected_bowling_types != "All":
        bowling_type_values = []
        if "RAP" in selected_bowling_types:
            bowling_type_values.append(1)
        if "LAP" in selected_bowling_types:
            bowling_type_values.append(2)
        filtered_data = filtered_data[filtered_data['BowlingTypeGroup'].isin(bowling_type_values)]
elif pace_or_spin == 'Spin':
    bowling_type_options = ["All", "RAO", "SLAO", "RALB", "LAC"]
    selected_bowling_types = input(f"Select Bowling Type Group ({', '.join(bowling_type_options)}): ")
    if selected_bowling_types != "All":
        bowling_type_values = []
        if "RAO" in selected_bowling_types:
            bowling_type_values.append(3)
        if "SLAO" in selected_bowling_types:
            bowling_type_values.append(4)
        if "RALB" in selected_bowling_types:
            bowling_type_values.append(5)
        if "LAC" in selected_bowling_types:
            bowling_type_values.append(6)
        filtered_data = filtered_data[filtered_data['BowlingTypeGroup'].isin(bowling_type_values)]
print(f"Data after bowling type group filter: {filtered_data.shape[0]} rows")

# Phase selection
phase_type = input("Select phase type (3Phase, 4Phase): ")
if phase_type == "3Phase":
    phase_options = ["All", 1, 2, 3]
    selected_phase = input(f"Select Phase ({', '.join(map(str, phase_options))}): ")
    if selected_phase != "All":
        filtered_data = filtered_data[filtered_data['Phase3idStar'] == int(selected_phase)]
elif phase_type == "4Phase":
    phase_options = ["All", 1, 2, 3, 4]
    selected_phase = input(f"Select Phase ({', '.join(map(str, phase_options))}): ")
    if selected_phase != "All":
        filtered_data = filtered_data[filtered_data['Phase4id'] == int(selected_phase)]
print(f"Data after phase filter: {filtered_data.shape[0]} rows")

# Display unique values in run type columns before filtering
run_type_columns = ['1s', '2s', '3s', '0s', 'Batwkts', '4s', '6s']
for col in run_type_columns:
    unique_values = filtered_data[col].unique()
    print(f"Unique values in {col}: {unique_values}")

# Run types filter
run_types = input("Select run types (comma-separated, e.g., '1s, 4s, 6s'): ").split(',')
run_types = [run_type.strip() for run_type in run_types]

# Filter based on any of the selected run types
if run_types:
    condition = None
    for run_type in run_types:
        print(f"Filtering for run type: {run_type}")
        if condition is None:
            condition = filtered_data[run_type] == 1
        else:
            condition |= filtered_data[run_type] == 1
    filtered_data = filtered_data[condition]

print(f"Data after run types filter: {filtered_data.shape[0]} rows")

# Ensure filtered_data is not empty before accessing its elements
if not filtered_data.empty:
    # Read the image corresponding to the batsman's batting type
    batsman_batting_type = filtered_data['StrikerBattingType'].iloc[0]
    image_path = image_paths[str(batsman_batting_type)]
    img = plt.imread(image_path)
    height, width, _ = img.shape

    plt.imshow(img)

    # Plot each ball in the filtered data
    for index, row in filtered_data.iterrows():
        ball_type = determine_ball_type(row)
        if ball_type:
            plot_balls(row['x'], row['y'], row['StrikerBattingType'], ball_type)

    plt.show()
else:
    print("No data available for the selected filters.")


Data after date filter: 3528 rows
Data after competition filter: 3528 rows
Data after batsman club filter: 624 rows
Data after match filter: 116 rows
Data after batsman filter: 4 rows
Data after pace/spin filter: 4 rows
Data after bowling type group filter: 4 rows
Data after phase filter: 4 rows
Unique values in 1s: [0]
Unique values in 2s: [0]
Unique values in 3s: [0]
Unique values in 0s: [1.]
Unique values in Batwkts: [0 1]
Unique values in 4s: [0]
Unique values in 6s: [0]
Filtering for run type: 4s
Filtering for run type: 6s
Data after run types filter: 0 rows
No data available for the selected filters.


In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

# Define the regions and conditions for plotting the balls
OldRegStartSS_Y = 189
OldRegStumpHeightSS_Y = 312
OldRegStumpLineSS_Y = 406
OldRegEndSS_Y = 446
OldRegStartSS_X = 144
OldRegEndSS_X = 462

RegStartSS_Y = 90
RegEndSS_Y = 467
RegStartSS_X = 300
RegEndSS_X = 781
RegStumpHeightSS_Y = 335
RegStumpLineSS_Y = 439

width = 0
height = 0

def ShortZoneXaxis(HeightX):
    if HeightX >= OldRegStartSS_X and HeightX <= OldRegEndSS_X:
        return (((RegEndSS_X - RegStartSS_X) / (OldRegEndSS_X - OldRegStartSS_X)) * (HeightX - OldRegStartSS_X)) + RegStartSS_X
    else:
        return None

def ShortZoneYaxis(HeightY):
    if HeightY >= OldRegStartSS_Y and HeightY <= OldRegStumpHeightSS_Y:
        return (((RegStumpHeightSS_Y - RegStartSS_Y) / (OldRegStumpHeightSS_Y - OldRegStartSS_Y)) * (HeightY - OldRegStartSS_Y)) + RegStartSS_Y
    elif HeightY > OldRegStumpHeightSS_Y and HeightY <= OldRegStumpLineSS_Y:
        return (((RegStumpLineSS_Y - RegStumpHeightSS_Y) / (OldRegStumpLineSS_Y - OldRegStumpHeightSS_Y)) * (HeightY - OldRegStumpHeightSS_Y)) + RegStumpHeightSS_Y
    elif HeightY > OldRegStumpLineSS_Y and HeightY <= OldRegEndSS_Y:
        return (((RegEndSS_Y - RegStumpLineSS_Y) / (OldRegEndSS_Y - OldRegStumpLineSS_Y)) * (HeightY - OldRegStumpLineSS_Y)) + RegStumpLineSS_Y
    else:
        return None

def plot_balls(x, y, StrikerBattingType, ball_type):
    XAxisValue_New = ShortZoneXaxis(x)
    YAxisValue_New = ShortZoneYaxis(y)
    if XAxisValue_New is not None and YAxisValue_New is not None:
        X = (XAxisValue_New / 1080 * width) - 7
        Y = (YAxisValue_New / 600 * height) - 13
        color = get_ball_color(ball_type)
        plt.scatter(X, Y, marker='o', color=color, label=ball_type)

def get_ball_color(ball_type):
    colors = {
        '1s': 'goldenrod',
        '2s': 'blue',
        '3s': 'green',
        '0s': 'black',
        'Batwkts': 'azure',
        '4s': 'darkblue',
        '6s': 'red'
    }
    return colors.get(ball_type, 'gray')  # Default color for unknown types

def determine_ball_type(row):
    if row['1s'] == 1:
        return '1s'
    elif row['2s'] == 1:
        return '2s'
    elif row['3s'] == 1:
        return '3s'
    elif row['0s'] == 1:
        return '0s'
    elif row['Batwkts'] == 1:
        return 'Batwkts'
    elif row['4s'] == 1:
        return '4s'
    elif row['6s'] == 1:
        return '6s'
    else:
        return None

image_paths = {
    '1': 'bee_r.jpg',
    '2': 'bee_l.jpg'
}

# Read the CSV file containing x, y, battingtype, StrikerName, and runs columns
data = pd.read_csv('NewData.csv')  # Update with your CSV file path

# Date filter
data['Date'] = pd.to_datetime(data['date'], dayfirst=True)
start_date = pd.to_datetime(input("Enter start date (YYYY-MM-DD): "))
end_date = pd.to_datetime(input("Enter end date (YYYY-MM-DD): "))
filtered_data = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)]
print(f"Data after date filter: {filtered_data.shape[0]} rows")

# Competition filter
competitions = list(filtered_data['CompName'].unique())
selected_competition = input(f"Select competition ({', '.join(competitions)}): ")
if selected_competition:
    filtered_data = filtered_data[filtered_data['CompName'] == selected_competition]
print(f"Data after competition filter: {filtered_data.shape[0]} rows")

# Batsman club filter
bat_club_names = list(map(str, filtered_data['battingclubid'].unique()))
selected_bat_club_name = input(f"Select the batsman's club id ({', '.join(bat_club_names)}): ")
if selected_bat_club_name:
    filtered_data = filtered_data[filtered_data['battingclubid'] == int(selected_bat_club_name)]
print(f"Data after batsman club filter: {filtered_data.shape[0]} rows")

# Match filter
match_ids = ['All'] + list(map(str, filtered_data['matchid'].unique()))
selected_match_id = input(f"Select Match ({', '.join(match_ids)}): ")
if selected_match_id != 'All':
    filtered_data = filtered_data[filtered_data['matchid'] == int(selected_match_id)]
print(f"Data after match filter: {filtered_data.shape[0]} rows")

# Batsman filter
batsman_names = ['All'] + list(filtered_data['StrikerName'].unique())
selected_batsman_name = input(f"Select the batsman's name ({', '.join(batsman_names)}): ")
if selected_batsman_name != 'All':
    filtered_data = filtered_data[filtered_data['StrikerName'] == selected_batsman_name]
print(f"Data after batsman filter: {filtered_data.shape[0]} rows")

# Pace or Spin filter
pace_or_spin = input("Select bowler type (Pace, Spin, Both): ")
if pace_or_spin == 'Pace':
    filtered_data = filtered_data[filtered_data['PaceorSpin'] == 1]
elif pace_or_spin == 'Spin':
    filtered_data = filtered_data[filtered_data['PaceorSpin'] == 2]
print(f"Data after pace/spin filter: {filtered_data.shape[0]} rows")

# Bowling Type Group filter
if pace_or_spin == 'Pace':
    bowling_type_options = ["All", "RAP", "LAP"]
    selected_bowling_types = input(f"Select Bowling Type Group ({', '.join(bowling_type_options)}): ")
    if selected_bowling_types != "All":
        bowling_type_values = []
        if "RAP" in selected_bowling_types:
            bowling_type_values.append(1)
        if "LAP" in selected_bowling_types:
            bowling_type_values.append(2)
        filtered_data = filtered_data[filtered_data['BowlingTypeGroup'].isin(bowling_type_values)]
elif pace_or_spin == 'Spin':
    bowling_type_options = ["All", "RAO", "SLAO", "RALB", "LAC"]
    selected_bowling_types = input(f"Select Bowling Type Group ({', '.join(bowling_type_options)}): ")
    if selected_bowling_types != "All":
        bowling_type_values = []
        if "RAO" in selected_bowling_types:
            bowling_type_values.append(3)
        if "SLAO" in selected_bowling_types:
            bowling_type_values.append(4)
        if "RALB" in selected_bowling_types:
            bowling_type_values.append(5)
        if "LAC" in selected_bowling_types:
            bowling_type_values.append(6)
        filtered_data = filtered_data[filtered_data['BowlingTypeGroup'].isin(bowling_type_values)]
print(f"Data after bowling type group filter: {filtered_data.shape[0]} rows")

# Phase selection
phase_type = input("Select phase type (3Phase, 4Phase): ")
if phase_type == "3Phase":
    phase_options = ["All", 1, 2, 3]
    selected_phase = input(f"Select Phase ({', '.join(map(str, phase_options))}): ")
    if selected_phase != "All":
        filtered_data = filtered_data[filtered_data['Phase3idStar'] == int(selected_phase)]
elif phase_type == "4Phase":
    phase_options = ["All", 1, 2, 3, 4]
    selected_phase = input(f"Select Phase ({', '.join(map(str, phase_options))}): ")
    if selected_phase != "All":
        filtered_data = filtered_data[filtered_data['Phase4idStar'] == int(selected_phase)]
print(f"Data after phase filter: {filtered_data.shape[0]} rows")

# Run types filter
run_type_columns = ['1s', '2s', '3s', '0s', 'Batwkts', '4s', '6s']
for col in run_type_columns:
    unique_values = filtered_data[col].unique()
    print(f"Unique values in {col}: {unique_values}")

run_types = input("Select run types (comma-separated, e.g., '1s, 4s, 6s'): ").split(',')
run_types = [run_type.strip() for run_type in run_types if run_type.strip() in run_type_columns]

if run_types:
    conditions = [filtered_data[run_type] == 1 for run_type in run_types]
    if conditions:
        combined_condition = conditions[0]
        for condition in conditions[1:]:
            combined_condition |= condition
        filtered_data = filtered_data[combined_condition]

print(f"Data after run types filter: {filtered_data.shape[0]} rows")

# Ensure filtered_data is not empty before accessing its elements
if not filtered_data.empty:
    # Read the image corresponding to the batsman's batting type
    batsman_batting_type = filtered_data['StrikerBattingType'].iloc[0]
    image_path = image_paths[str(batsman_batting_type)]
    img = plt.imread(image_path)
    height, width, _ = img.shape

    plt.imshow(img)

    # Plot each ball in the filtered data
    for index, row in filtered_data.iterrows():
        ball_type = determine_ball_type(row)
        if ball_type:
            plot_balls(row['HeightX'], row['HeightY'], row['StrikerBattingType'], ball_type)

    plt.show()
else:
    print("No data available for the selected filters.")

#finally working, ufffffff!!!!
#finally working, ufffffff!!!!
#finally working, ufffffff!!!!
#finally working, ufffffff!!!!
#finally working, ufffffff!!!!
#finally working, ufffffff!!!!
#finally working, ufffffff!!!!
#finally working, ufffffff!!!!
#finally working, ufffffff!!!!

Data after date filter: 3528 rows
Data after competition filter: 3528 rows
Data after batsman club filter: 640 rows
Data after match filter: 131 rows
Data after batsman filter: 0 rows
Data after pace/spin filter: 0 rows
Data after bowling type group filter: 0 rows
Data after phase filter: 0 rows
Unique values in 1s: []
Unique values in 2s: []
Unique values in 3s: []
Unique values in 0s: []
Unique values in Batwkts: []
Unique values in 4s: []
Unique values in 6s: []
Data after run types filter: 0 rows
No data available for the selected filters.
